In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
from src.utils import isWorkday, estEndDate, isParentCompleted, isWeekend, calcLength, delay, assessWeather, isHeavyWeather
import pyodbc
import random
from tqdm import tqdm

In [2]:
server = 'LAPTOP-2NSE0JH1\SQLEXPRESS'
database = 'dummy'

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

try:
    conn = pyodbc.connect(conn_str)

    cursor = conn.cursor()

    task_query = "SELECT * FROM Task"
    tasks = pd.read_sql(task_query, conn)

    project_query = "SELECT * FROM Project"
    projects = pd.read_sql(project_query, conn)

    conn.close()

except pyodbc.Error as e:
    print("Error connecting to SQL Server:", e)

C:\Users\Acer\AppData\Local\Temp\ipykernel_19648\2839744962.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tasks = pd.read_sql(task_query, conn)
C:\Users\Acer\AppData\Local\Temp\ipykernel_19648\2839744962.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  projects = pd.read_sql(project_query, conn)


In [3]:
tasks['StartDate'].min(),tasks['StartDate'].max()

(datetime.date(2016, 1, 1), datetime.date(2020, 6, 19))

In [4]:
for pid in tasks['ProjectID'].unique():
    curr = tasks[tasks['ProjectID']==pid]
    print(pid, (curr['EndDate'].max()-curr['StartDate'].min()).days, len(curr))

1 402 115
2 412 111
3 413 135
4 313 136
5 483 120
6 441 111
7 658 115
8 343 140
9 287 93
10 498 96
11 321 116
12 260 138
13 279 144
14 550 136
15 340 149
16 328 109
17 398 123
18 322 93
19 468 141
20 327 110
21 334 131
22 523 97
23 332 98
24 369 141
25 579 125
26 501 150
27 735 140
28 699 109
29 293 100
30 634 115
31 335 130
32 335 143
33 288 85
34 240 132
35 306 147
36 355 134
37 594 111
38 389 150
39 716 132
40 466 93
41 297 142
42 422 144
43 577 108
44 466 135
45 595 150
46 323 149
47 332 98
48 735 119
49 348 132
50 404 110
51 553 150
52 234 86
53 267 129
54 299 139
55 451 139
56 355 120
57 570 87
58 272 99
59 466 117
60 600 134
61 251 86
62 252 130
63 316 132
64 621 112
65 308 92
66 240 131
67 649 91
68 536 141
69 435 93
70 249 125
71 356 95
72 414 121
73 591 135
74 298 139
75 343 113
76 656 110
77 582 118
78 289 109
79 850 138
80 382 148
81 362 128
82 357 145
83 309 139
84 431 87
85 393 124
86 248 101
87 265 129
88 409 140
89 321 130
90 225 89
91 476 125
92 815 117
93 508 81
94 31

In [5]:
# tasks = pd.read_csv('data/10_2018/task.csv')
# projects = pd.read_csv('data/10_2018/project.csv')

In [6]:
weather_all = pd.read_csv('data/weather/weather_all.csv')
weather_all['datetime'] = pd.to_datetime(weather_all['datetime'])
weather_all = weather_all.set_index('datetime')

In [7]:
tasks['ParentTaskID'] = tasks['ParentTaskID'].astype('Int64') 
tasks['StartDate'] = pd.to_datetime(tasks['StartDate'])
tasks['EndDate'] = pd.to_datetime(tasks['EndDate'])
tasks['ActualStartDate'] = pd.to_datetime(tasks['ActualStartDate'])
tasks['ActualEndDate'] = pd.to_datetime(tasks['ActualEndDate'])
tasks['WeatherAssessment'] = tasks.apply(lambda x: assessWeather(x['StartDate'],projects.loc[projects['ID']==x['ProjectID']]['Workday'].iloc[0], weather_all),axis=1)
tasks['TaskLength'] = calcLength(tasks['ParentTaskID'])

In [10]:
# tasks.to_csv('data/100_2016/task.csv', index=False)
# projects.to_csv('data/100_2016/project.csv', index=False)

In [11]:
curr_date = tasks['StartDate'].min()
task_report = []
project_report = []

total_tasks = len(tasks)
pbar = tqdm(total=total_tasks, desc="Progress")

while ~tasks['Status'].eq('Completed').all():
    task_today = tasks[(tasks['StartDate'] <= curr_date) & (tasks['Status']!='Completed')]['ID'].tolist()
    heavy_weather = isHeavyWeather(curr_date, weather_all)

    for idx in task_today:
        task = tasks.loc[tasks['ID']==idx].iloc[0]
        workday = projects.loc[projects['ID'] == task['ProjectID']].iloc[0]['Workday']
        if isParentCompleted(task, tasks) and isWorkday(curr_date, workday):
            if task['Status'] == 'Not Started':
                task['ActualStartDate'] = str(curr_date)
                task['Status'] = 'On Progress'
                
            if delay(task, task_today, curr_date, heavy_weather):
                task['Progress'] += 0
            else:
                task['Progress'] += 1
                
            if task['Progress'] >= task['Duration']:
                task['ActualEndDate'] = curr_date
                task['Status'] = 'Completed'
                pbar.update(1)
            
            if curr_date > task['EndDate'] and task['Status'] == 'On Progress':
                task['Status'] = 'Delayed'
                task['Priority'] = 'Critical'
            
            tasks.loc[tasks['ID']==idx] = task.values
            
            task_report.append({
                'Date': curr_date,
                'ID': task['ID'],
                'Name': task['Name'],
                'StartDate': task['StartDate'],
                'EndDate': task['EndDate'],
                'Cost': task['Cost'],
                'Priority': task['Priority'],
                'Progress': task['Progress'],
                'ProjectID' : task['ProjectID'],
                'Status': task['Status'],
                'Duration': task['Duration'],
                'Trade' : task['Trade'],
                'TaskLength' : task['TaskLength'],
                'IsBadWeather' : heavy_weather,
                'WeatherAssessment' : task['WeatherAssessment'],
                'ActualStartDate': task['ActualStartDate'],
                'ActualEndDate': task['ActualEndDate']
            })
            
    for pid in projects['ID'].tolist():
        project_task = tasks[tasks['ProjectID']==pid]
        if ~project_task['Status'].eq('Not Started').all() and ~project_task['Status'].eq('Completed').all():
            project_report.append({
                'Date' : curr_date,
                'ProjectID' : pid,
                'TotalTask' : len(project_task),
                'StartedTask' : len(project_task[project_task['Status']!='Not Started']),
                'OnGoingTask' : len(project_task[(project_task['Status']!='Not Started') & (project_task['Status']!='Completed')]),
                'DelayedTask' : len(project_task[(project_task['Status']=='Delayed')]),
                'CompletedTask' : len(project_task[(project_task['Status']=='Completed')]),
                'WorkDay' : project_task[(project_task['Status']!='Not Started')]['Progress'].sum(),
                'TotalSpent' : project_task[(project_task['Status']=='Completed')]['Cost'].sum(),
                'IsBadWeather' : heavy_weather
            })

    curr_date += timedelta(days=1)

task_reports = pd.DataFrame(task_report)
project_reports = pd.DataFrame(project_report)
task_reports['ActualEndDate'] = task_reports.groupby('ID')['ActualEndDate'].bfill()
# task_ns = tasks.copy()
# task_ns['ActualStartDate'] = task_ns['StartDate']
# task_ns['Date'] = task_ns['ActualStartDate']
# task_ns['Priority'] = task_ns.apply(lambda x: 'Critical' if pd.isna(x['ParentTaskID']) else 'Normal', axis=1)
# task_ns['Progress'] = 0
# task_ns['Status'] = 'Not Started'
# task_ns['IsBadWeather'] = 0
# task_ns.drop(['ParentTaskID', 'AssigneeID', 'CreateDate'],axis=1, inplace=True)
# task_reports = pd.concat([task_reports, task_ns]).reset_index(drop=True)

project_dates = tasks.groupby('ProjectID').agg({'StartDate': 'min', 'EndDate': 'max', 'ActualStartDate':'min','ActualEndDate':'max'}).reset_index()
project_dates['WeatherAssessment'] = project_dates.apply(lambda x: assessWeather(x['StartDate'],projects.loc[projects['ID']==x['ProjectID']]['Workday'].iloc[0],weather_all),axis=1)
project_reports = pd.merge(project_reports, project_dates, on='ProjectID', how='left')
# project_ns = project_reports.copy()
# project_ns = project_reports.groupby('ProjectID').agg({'TotalTask':'max','StartDate':'max','EndDate':'max','ActualStartDate':'max','ActualEndDate':'max','WeatherAssessment':'mean'}).reset_index()
# project_ns['Date'] = project_ns['ActualStartDate']
# project_ns[['StartedTask','OnGoingTask','DelayedTask','CompletedTask', 'WorkDay', 'TotalSpent','IsBadWeather']] = 0
# project_reports = pd.concat([project_reports,project_ns]).reset_index(drop=True)

print(f'Complete all task at {curr_date}')
pbar.close()

Progress: 100%|██████████| 12099/12099 [09:53<00:00, 20.40it/s]

Complete all task at 2020-06-24 00:00:00


In [18]:
task_reports.to_csv('data/100_2016/task_report.csv',index=False)
project_reports.to_csv('data/100_2016/project_report.csv',index=False)